In [ ]:
import json
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_simplex"

In [ ]:
# Load nbS1-O1 circuit with connectivity matrix
circuit = obi.Circuit(name="nbS1-O1",
                      path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"),
                      matrix_path=os.path.join(cmat_root, "nbS1-O1", "connectivity_matrix.h5"))
display(circuit.connectivity_matrix.matrix)

In [ ]:
# nbS1-O1 - Central neurons in connectivity
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
# df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")  # Don't used here

# Load HEXi connectivity matrix
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

# Load ID mapping w.r.t. parent circuit nbS1-O1
id_map_file = os.path.join(input_root, f"{base_name}-{hex_sel.upper()}", "id_mapping.json")
with open(id_map_file, "r") as f:
    id_map = json.load(f)
assert np.array_equal(id_map["S1nonbarrel_neurons"]["new_id"], ntab.index.values)

nid_selection = []  # Selected neuron IDs for nbS1-O1 circuit
for _lay in layers:
    # opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    opt_idx = 0  # Always take first one from list, which is the most central neuron
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    nid_selection.append(id_map["S1nonbarrel_neurons"]["old_id"][node_id])

# Check selection
cmat = circuit.connectivity_matrix
sel_tab = cmat.vertices.loc[nid_selection]
assert np.array_equal(sel_tab["layer"].astype(int), layers)
display(sel_tab)

In [ ]:
# Set up simplex neuron sets
N = 10  # Max. number of neurons
# dim = 10  # Top dim (may be actually 4, 5 or 6)
dim = 2  # Triplets
# simplex_pos = "source"
simplex_pos = "target"
simplex_neuron_sets = obi.SimplexNeuronSet(
    central_neuron_id=nid_selection,
    dim=dim,
    central_neuron_simplex_position=simplex_pos,
    subsample=True,
    seed=0,
    n_count_max=N,
    simplex_type="directed",
    property_filter=obi.NeuronPropertyFilter(),
)

virtual_sources_to_ignore = ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", )
output_path = os.path.join(output_root, f"dim{dim}-{simplex_pos}")

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    neuron_set=simplex_neuron_sets,
)

coupled_scan = obi.CoupledScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [ ]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
# coupled_scan.coordinate_instances(display=True)

In [ ]:
coupled_scan.execute()
obi.run_tasks_for_generated_scan(coupled_scan)

In [ ]:
# Check circuits
from bluepysnap import Circuit

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['S1nonbarrel_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"L{layers[_idx]}: {nodes.size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {', '.join(mt_hist)}")
